In [1]:
%load_ext autoreload
%autoreload 2

In [6]:
#carga de datasets
from utils.DatasetStorage import Dataset
from utils.paths import *

#clasificadores
from utils.clasificacion import *

#adaptacion
from utils.adaptacion import pca_grid_search

#otros
import os
import numpy as np
import pandas as pd
import timeit
from sklearn.externals import joblib

tipo = pruebas[3]
dataset_name = datasets[0]
dims = dimensions[dataset_name]

## Pruebas con el dataset de Amazon

In [3]:
print tipo
print dataset_name
print dims
print data_path

pca
amazon
1000
data


In [4]:
# cargando dataset Amazon
dataset_path = os.path.join(data_path, dataset_name+'.pkl')
dataset_object = Dataset().load(dataset_path)

dataset_object.split_dataset(test_size=0.2)

labeled = dataset_object.labeled
domains = dataset_object.domains

Dataset already splitted


# Adaptación

## Creación de modelos de adaptación.

Por cada par de dominios se entrenan distintos modelos según los parámetros establecidos.
Al entrenar cada modelo, este es inmediatamente probado, siendo almacenado o descartado según su desempeño.

Cada modelo es guardado en la ruta: models/amazon/pca/me2\_[dominio\_fuente]_[dominio\_objetivo].pkl

### Creación de modelos de adaptación.

In [8]:
models_paths = os.path.join(models_path, dataset_name, tipo, 'me2_models_paths.pkl')

parameters = {
    'n_components': [int(dims/2), int(dims/3), int(dims/4)],
}

i = 1

#por cada par se entrena un adaptador y se guarda el que mejor resultados entrega
print "Adaptando entre:"
for src in domains:
    for tgt in domains:
        if src is not tgt:
            print "%d) %s - %s" % (i, src, tgt)
            X_src = np.asarray(labeled[src]['X_tr'].todense())
            y_src = np.asarray(labeled[src]['y_tr'].todense()).argmax(axis=1)

            X_tgt = np.asarray(labeled[tgt]['X_tr'].todense())
            
            X_train = np.concatenate((X_src, X_tgt))
            
            best_model, score = pca_grid_search(X_train, X_src, y_src, parameters, n_jobs=4)
            print "\tMejor modelo: %.3f" % score
            
            folder_path = os.path.join(models_path, dataset_name, tipo)
            prefix = "me2_" + src + "_" + tgt + ".pkl"
            
            best_model_save_path = os.path.join(folder_path, prefix)
            print "\tGuardando modelo en %s\n" % best_model_save_path
            joblib.dump(best_model, best_model_save_path)
            i = i+1

print "\nAdaptaciones completadas"

Adaptando entre:
kitchen - dvd
	n_components: 500
0.872285537124
	n_components: 333


/home/francisco/miniconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/svm/base.py:220: ConvergenceWarning: Solver terminated early (max_iter=50000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/home/francisco/miniconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/svm/base.py:220: ConvergenceWarning: Solver terminated early (max_iter=50000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/home/francisco/miniconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/svm/base.py:220: ConvergenceWarning: Solver terminated early (max_iter=50000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/home/francisco/miniconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/svm/base.py:220: ConvergenceWarning: Solver terminated early (max_iter=50000).  Consider pre-processing your d

0.883858149366
	n_components: 250
0.878536203654
	Mejor modelo: 0.884
	Guardando modelo en models/amazon/pca/me2_kitchen_dvd.pkl
kitchen - electronics
	n_components: 500
0.878607405471
	n_components: 333
0.883574087672
	n_components: 250
0.891560590898
	Mejor modelo: 0.892
	Guardando modelo en models/amazon/pca/me2_kitchen_electronics.pkl
kitchen - books
	n_components: 500
0.874367396096
	n_components: 333
0.881354976482
	n_components: 250
0.895026200212
	Mejor modelo: 0.895
	Guardando modelo en models/amazon/pca/me2_kitchen_books.pkl
dvd - kitchen
	n_components: 500
0.800363662277
	n_components: 333
0.821752387178
	n_components: 250
0.822691725629
	Mejor modelo: 0.823
	Guardando modelo en models/amazon/pca/me2_dvd_kitchen.pkl
dvd - electronics
	n_components: 500
0.804796969303
	n_components: 333
0.829177261868
	n_components: 250
0.81982251214
	Mejor modelo: 0.829
	Guardando modelo en models/amazon/pca/me2_dvd_electronics.pkl
dvd - books
	n_components: 500
0.799475634424
	n_components:

## Pruebas de clasificación

In [9]:
df = pd.DataFrame(columns=dataframe_columns)

i=1
tareas = len(domains)*(len(domains)-1)

# por cada par posible para adaptar
for src in domains:
    for tgt in domains:
        if src is not tgt:
            print "Tarea %d de %d" % (i, tareas)
            
            #baseline in-domain error
            #e_b(T,T)
            #entrenado en dominio tgt y probado en dominio tgt
            X_tr = np.asarray(labeled[tgt]['X_tr'].todense())
            y_tr = np.asarray(labeled[tgt]['y_tr'].todense()).argmax(axis=1)
            
            X_ts = np.asarray(labeled[tgt]['X_ts'].todense())
            y_ts = np.asarray(labeled[tgt]['y_ts'].todense()).argmax(axis=1)
            
            # se crean las rutas para cargar o crear los modelos
            model_name = "%s.pkl" % (tgt)
            model_path = os.path.join(models_path, dataset_name, "indomain", model_name)
            
            #Se realiza una clasificacion, estimando los parametros mediante cv
            svc = load_best_score(model_path, X_tr, y_tr)
            b_error = 1-svc.score(X_ts, y_ts)
            
            #############
            #### PCA ####
            #############
            # se adaptan los dominios usando PCA
            print "Adaptando dominios..."
            folder_path = os.path.join(models_path, dataset_name, tipo)
            prefix = "me2_" + src + "_" + tgt + ".pkl"
            
            best_model_save_path = os.path.join(folder_path, prefix)
            
            best_pca = joblib.load(best_model_save_path)
            
            
            X_tr = np.asarray(labeled[src]['X_tr'].todense())
            X_tr_a = best_pca.transform(X_tr)
            y_tr = np.asarray(labeled[src]['y_tr'].todense()).argmax(axis=1)
            
            X_ts = np.asarray(labeled[tgt]['X_ts'].todense())
            X_ts_a = best_pca.transform(X_ts)
            y_ts = np.asarray(labeled[tgt]['y_ts'].todense()).argmax(axis=1)
            
            clf = get_best_score(X_tr_a, y_tr, classifier='SVC', n_jobs=4)
            t_error = 1-clf.score(X_ts_a, y_ts)
            
            
            # transfer loss t
            # t_error - b_error
            t_loss = t_error - b_error
            
            tarea = src[0]+'->'+tgt[0]
            df.loc[i] = ['PCA',tarea,src,tgt,b_error*100,t_error*100, t_loss*100]
            
            i += 1
            
print "\nPruebas completadas."

Tarea 1 de 12
Cargando modelo existente.
Adaptando dominios...
Tarea 2 de 12
Cargando modelo existente.
Adaptando dominios...
Tarea 3 de 12
Cargando modelo existente.
Adaptando dominios...
Tarea 4 de 12
Cargando modelo existente.
Adaptando dominios...
Tarea 5 de 12
Cargando modelo existente.
Adaptando dominios...
Tarea 6 de 12
Cargando modelo existente.
Adaptando dominios...
Tarea 7 de 12
Cargando modelo existente.
Adaptando dominios...
Tarea 8 de 12
Cargando modelo existente.
Adaptando dominios...
Tarea 9 de 12
Cargando modelo existente.
Adaptando dominios...
Tarea 10 de 12
Cargando modelo existente.
Adaptando dominios...
Tarea 11 de 12
Cargando modelo existente.
Adaptando dominios...
Tarea 12 de 12
Cargando modelo existente.
Adaptando dominios...

Pruebas completadas.


In [10]:
df

,Adaptacion,Tarea,Fuente,Objetivo,Baseline error,Transfer error,Transfer loss
1,PCA,k->d,kitchen,dvd,26.25,37.803445,11.553445
2,PCA,k->e,kitchen,electronics,23.50,15.067877,-8.432123
3,PCA,k->b,kitchen,books,25.50,34.395860,8.895860
4,PCA,d->k,dvd,kitchen,20.75,20.775519,0.025519
5,PCA,d->e,dvd,electronics,23.50,29.223231,5.723231
6,PCA,d->b,dvd,books,25.50,25.365634,-0.134366
7,PCA,e->k,electronics,kitchen,20.75,11.370284,-9.379716
8,PCA,e->d,electronics,dvd,26.25,28.778219,2.528219
9,PCA,e->b,electronics,books,25.50,27.993200,2.493200
10,PCA,b->k,books,kitchen,20.75,21.283032,0.533032


In [11]:
new_scores_path = os.path.join(scores_path,dataset_name, tipo, "me2_%d.csv" % (dims))

print "Guardando en %s" % new_scores_path
df.to_csv(new_scores_path, columns=df.columns)
print "Resultados guardados."

Guardando en scores/amazon/pca/me2_1000.csv
Resultados guardados.
